# Primera red neuronal

Una red neuronal, es una arquitectura de varias capas/módulos que realizan operaciones con los datos.
En PyTorch se utilizan la *torch.nn*, el comopente báse para construir una red neuronal.
Cada módulo se subclassifica con *nn.Module*.
Una red neuronal en PyTorch trata de un módulo *nn.Module* que dentro contiene componentes *torch.nn*, por ejemplo ***nn.Linear***

Esta estructura anidada permite construir y gestionar arquitecturas complejas fácilmente.

#### Importaciones y Cuda

In [1]:
# Importar librerias
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# Activar Cuda
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Usando el servicio: {device}")

Usando el servicio: cuda


#### Constuyendo la red

El primer paso es crear una subclase del modulo *nn.Module* e inicializar las capas que queramos en el *__init __*
Cada clase *nn.Module* implementa las operaciones sobre los datos de entrada en el método *forward*.

In [3]:
class NeuralNetwork(nn.Module): # Hereda de la clase nn.Module
    def __init__(self):
        super().__init__() # Llamada al constructor de la clase padre
        self.flatten = nn.Flatten() # Convierte la imagen 28x28 en un vector de 784 elementos
        self.linear_relu_stack = nn.Sequential( # Crea una secuencia de capas lineales y relu
            nn.Linear(28*28, 512), # Capa lineal densa de 28x28 elementos a 512 elementos (input)
            nn.ReLU(), # Función de activación ReLU
            nn.Linear(512, 512), # Capa lineal densa
            nn.ReLU(), # Función de activación ReLU
            nn.Linear(512, 10), # Capa lineal densa (output)
        )

    def forward(self, x): 
        x = self.flatten(x) # Aplicar capa  
        logits = self.linear_relu_stack(x) # Aplicar secuencia capa
        return logits

Ahora creamos la instancia de la clase *NeuralNetwork* y le indicamos que se ejecute en el device seleccionado.

In [4]:
model = NeuralNetwork().to(device) # Crea la red y la pasa al dispositivo
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Cone sto, ya tendremos el modelo creado en la variable *model*. Para ejecutarlo,  **NO utilizamos el método *forward***, para ejecutarlo, utilizamos directamente la variable *model*, pasandole como parametro los datos de entrada.

In [7]:
X = torch.rand(1, 28, 28, device=device) # Crea un tensor aleatorio de 28x28 elementos
logits = model(X) # Calcula las probabilidades de las clases (Ejecutar el modelo)
pred_probab = nn.Softmax(dim=1)(logits) # Calcula la probabilidad de la clase
y_pred = pred_probab.argmax(1) # Obtiene la clase con la probabilidad mas alta
print(f"Predicted class: {y_pred}") # Imprime la clase predicha

Predicted class: tensor([9], device='cuda:0')


#### Ver parámetros del modelo

Aun que todavía no conocemos los detalles internos del modelo, podemos visualizar los parametros internos con el método *parameters()* o *named_parameters()*

In [13]:
print(f"Model structure: {model}\n\n") # Imprime la estructura de la red

for name, param in model.named_parameters(): # Imprime los nombres y valores de los parametros
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0032,  0.0327, -0.0321,  ..., -0.0317, -0.0034, -0.0061],
        [-0.0047, -0.0036,  0.0021,  ..., -0.0208,  0.0280,  0.0140]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0293, 0.0254], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0295,  0.0231,  0.0059,  ...,  0.0147, -0.0359,  0.0173],
        [ 0.0441,  0.0224, -0.0010,  ...,  0.0359, -0.0034, -0.0375]],
       device='cuda:0', grad_fn=<Slic